PyCitySchools Analysis Summary
As the Chief Data Scientist for the district, this analysis provided a comprehensive view of school performance across key indicators. By aggregating data at both the district and school level, we identified trends related to school size, budget, and type.

Key Takeaways
1. Charter Schools Outperform District Schools
Charter schools consistently achieved higher average math and reading scores, as well as higher percentages of students passing both subjects. Despite typically having lower budgets, charter schools had the highest overall passing rates, suggesting that spending alone doesn't determine success.

2. Smaller Schools Perform Better
Schools with fewer than 1,000 students performed significantly better than larger schools:

Average math score: 83.8 vs. 77.7

% Overall Passing: 89.8% for small schools vs. 58.3% for large schools

This may reflect more individualized attention, better teacher-student ratios, or less administrative complexity.

In [44]:
# Import dependencies
import pandas as pd
from pathlib import Path

In [45]:
# Load data
schools_df = pd.read_csv("Resources/schools_complete.csv")
students_df = pd.read_csv("Resources/students_complete.csv")

In [46]:
# Preview data
print(schools_df.head())
print(students_df.head())

   School ID            school_name      type  size   budget
0          0      Huang High School  District  2917  1910635
1          1   Figueroa High School  District  2949  1884411
2          2    Shelton High School   Charter  1761  1056600
3          3  Hernandez High School  District  4635  3022020
4          4    Griffin High School   Charter  1468   917500
   Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  
0             66          79  
1             94          61  
2             90          60  
3             67          58  
4             97          84  


In [47]:
# Merge the two datasets on the school name
school_data_complete = pd.merge(students_df, schools_df, how="left", on=["school_name"])
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [48]:
# Calculate values
total_schools = schools_df["school_name"].nunique()
total_students = students_df["Student ID"].count()
total_budget = schools_df["budget"].sum()
average_math_score = students_df["math_score"].mean()
average_reading_score = students_df["reading_score"].mean()

# Calculate % passing math, reading, and overall
passing_math = school_data_complete[school_data_complete["math_score"] >= 70].count()["student_name"]
passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].count()["student_name"]
overall_passing = school_data_complete[
    (school_data_complete["math_score"] >= 70) & 
    (school_data_complete["reading_score"] >= 70)
].count()["student_name"]

# Percentages
percent_passing_math = passing_math / total_students * 100
percent_passing_reading = passing_reading / total_students * 100
percent_overall_passing = overall_passing / total_students * 100

# Create summary DataFrame
district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [percent_passing_math],
    "% Passing Reading": [percent_passing_reading],
    "% Overall Passing": [percent_overall_passing]
})

district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [49]:
# Group by school
school_grouped = school_data_complete.groupby(["school_name"])

# School type
school_types = schools_df.set_index("school_name")["type"]

# Total students per school
per_school_counts = school_grouped["Student ID"].count()

# Total school budget
per_school_budget = schools_df.set_index("school_name")["budget"]

# Per student budget
per_student_budget = per_school_budget / per_school_counts

# Average math and reading scores
per_school_math = school_grouped["math_score"].mean()
per_school_reading = school_grouped["reading_score"].mean()

# % Passing math
per_school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["student_name"].count() / per_school_counts * 100

# % Passing reading
per_school_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["student_name"].count() / per_school_counts * 100

# % Overall passing (both math & reading)
overall_passing_rate = school_data_complete[
    (school_data_complete["math_score"] >= 70) & 
    (school_data_complete["reading_score"] >= 70)
].groupby("school_name")["student_name"].count() / per_school_counts * 100

# Create summary DataFrame
per_school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_student_budget,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": overall_passing_rate
})

# Optional: Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary["Average Math Score"] = per_school_summary["Average Math Score"].map("{:.1f}".format)
per_school_summary["Average Reading Score"] = per_school_summary["Average Reading Score"].map("{:.1f}".format)
per_school_summary["% Passing Math"] = per_school_summary["% Passing Math"].map("{:.1f}%".format)
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].map("{:.1f}%".format)
per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].map("{:.1f}%".format)

# Show it
per_school_summary.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.0,81.0,66.7%,81.9%,54.6%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94.1%,97.0%,91.3%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,66.0%,80.7%,53.2%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1,80.7,68.3%,79.3%,54.3%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93.4%,97.1%,90.6%


In [50]:
# Sort schools in descending order
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)

# Display top 5
top_schools.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94.1%,97.0%,91.3%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.4,83.8,93.3%,97.3%,90.9%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93.4%,97.1%,90.6%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.3,84.0,93.9%,96.5%,90.6%
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6%,95.9%,90.5%


In [51]:
# Sort schools in ascending order
bottom_schools = per_school_summary.sort_values("% Overall Passing", ascending=True)

# Display bottom 5
bottom_schools.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8,80.7,66.4%,80.2%,53.0%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,66.0%,80.7%,53.2%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.3,80.9,66.8%,80.9%,53.5%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6,81.2,65.7%,81.3%,53.5%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.1,81.0,66.1%,81.2%,53.5%


In [52]:
# Separate data by grade
ninth_graders = school_data_complete[school_data_complete["grade"] == "9th"]
tenth_graders = school_data_complete[school_data_complete["grade"] == "10th"]
eleventh_graders = school_data_complete[school_data_complete["grade"] == "11th"]
twelfth_graders = school_data_complete[school_data_complete["grade"] == "12th"]

# Group by school and calculate average math score
ninth_grade_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_grade_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_grade_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_grade_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Combine into a single DataFrame
math_scores_by_grade = pd.DataFrame({
    "9th": ninth_grade_math_scores,
    "10th": tenth_grade_math_scores,
    "11th": eleventh_grade_math_scores,
    "12th": twelfth_grade_math_scores
})

# Optional: Round for presentation
math_scores_by_grade = math_scores_by_grade.round(1)

# Display
math_scores_by_grade.head()


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4


In [53]:
# Group by school and calculate average reading score by grade
ninth_grade_reading_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_grade_reading_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_grade_reading_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_grade_reading_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()

# Combine into a single DataFrame
reading_scores_by_grade = pd.DataFrame({
    "9th": ninth_grade_reading_scores,
    "10th": tenth_grade_reading_scores,
    "11th": eleventh_grade_reading_scores,
    "12th": twelfth_grade_reading_scores
})

# Optional: Round for presentation
reading_scores_by_grade = reading_scores_by_grade.round(1)

# Display
reading_scores_by_grade.head()


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0


In [54]:
# Remove dollar signs from budget column (if not already numeric)
# Only run this if your values still have "$"
# per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].replace('[\$,]', '', regex=True).astype(float)

# Create spending bins and labels
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Convert per student budget to numeric (if needed)
per_school_summary["Per Student Budget (Num)"] = per_school_summary["Per Student Budget"].replace(r'[\$,]', '', regex=True).astype(float)


# Bin the data
per_school_summary["Spending Ranges (Per Student)"] = pd.cut(
    per_school_summary["Per Student Budget (Num)"],
    bins=spending_bins,
    labels=spending_labels
)


In [55]:
# Convert formatted columns back to numeric (strip % and $)
per_school_summary["Average Math Score"] = per_school_summary["Average Math Score"].astype(float)
per_school_summary["Average Reading Score"] = per_school_summary["Average Reading Score"].astype(float)
per_school_summary["% Passing Math"] = per_school_summary["% Passing Math"].replace('%','', regex=True).astype(float)
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].replace('%','', regex=True).astype(float)
per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].replace('%','', regex=True).astype(float)


In [56]:
# Group by spending range
spending_math_scores = per_school_summary.groupby("Spending Ranges (Per Student)")["Average Math Score"].mean()
spending_reading_scores = per_school_summary.groupby("Spending Ranges (Per Student)")["Average Reading Score"].mean()
spending_passing_math = per_school_summary.groupby("Spending Ranges (Per Student)")["% Passing Math"].mean()
spending_passing_reading = per_school_summary.groupby("Spending Ranges (Per Student)")["% Passing Reading"].mean()
spending_overall_passing = per_school_summary.groupby("Spending Ranges (Per Student)")["% Overall Passing"].mean()



/var/folders/21/0z_b5bn93sx2s8t9cpmzsyl40000gn/T/ipykernel_48574/1265413144.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = per_school_summary.groupby("Spending Ranges (Per Student)")["Average Math Score"].mean()
/var/folders/21/0z_b5bn93sx2s8t9cpmzsyl40000gn/T/ipykernel_48574/1265413144.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = per_school_summary.groupby("Spending Ranges (Per Student)")["Average Reading Score"].mean()
/var/folders/21/0z_b5bn93sx2s8t9cpmzsyl40000gn/T/ipykernel_48574/1265413144.py:4: FutureWarning: The default of observed=False is 

In [57]:
# Create size bins and labels
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Convert student count to numeric if it isn't already
per_school_summary["Total Students (Num)"] = per_school_summary["Total Students"].astype(int)

# Bin the schools by size
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students (Num)"],
                                           bins=size_bins,
                                           labels=size_labels)


In [58]:
size_math_scores = per_school_summary.groupby("School Size")["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby("School Size")["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby("School Size")["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby("School Size")["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby("School Size")["% Overall Passing"].mean()

type_summary[["% Passing Math", "% Passing Reading", "% Overall Passing"]] = type_summary[
    ["% Passing Math", "% Passing Reading", "% Overall Passing"]
].round(1).astype(str) + "%"



/var/folders/21/0z_b5bn93sx2s8t9cpmzsyl40000gn/T/ipykernel_48574/1426792183.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = per_school_summary.groupby("School Size")["Average Math Score"].mean()
/var/folders/21/0z_b5bn93sx2s8t9cpmzsyl40000gn/T/ipykernel_48574/1426792183.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_school_summary.groupby("School Size")["Average Reading Score"].mean()
/var/folders/21/0z_b5bn93sx2s8t9cpmzsyl40000gn/T/ipykernel_48574/1426792183.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a 

In [59]:
# Create the summary DataFrame
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})

# Format the DataFrame
size_summary = size_summary.round(1)
size_summary[["% Passing Math", "% Passing Reading", "% Overall Passing"]] = size_summary[
    ["% Passing Math", "% Passing Reading", "% Overall Passing"]
].applymap("{:.1f}%".format)

# Display
size_summary

type_summary[["% Passing Math", "% Passing Reading", "% Overall Passing"]] = type_summary[
    ["% Passing Math", "% Passing Reading", "% Overall Passing"]
].round(1).astype(str) + "%"


/var/folders/21/0z_b5bn93sx2s8t9cpmzsyl40000gn/T/ipykernel_48574/2531269238.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ].applymap("{:.1f}%".format)


In [60]:
# Group by school type
type_math_scores = per_school_summary.groupby("School Type")["Average Math Score"].mean()
type_reading_scores = per_school_summary.groupby("School Type")["Average Reading Score"].mean()
type_passing_math = per_school_summary.groupby("School Type")["% Passing Math"].mean()
type_passing_reading = per_school_summary.groupby("School Type")["% Passing Reading"].mean()
type_overall_passing = per_school_summary.groupby("School Type")["% Overall Passing"].mean()


In [61]:
# Create summary DataFrame
type_summary = pd.DataFrame({
    "Average Math Score": type_math_scores,
    "Average Reading Score": type_reading_scores,
    "% Passing Math": type_passing_math,
    "% Passing Reading": type_passing_reading,
    "% Overall Passing": type_overall_passing
})

# Format and display
type_summary = type_summary.round(1)
type_summary[["% Passing Math", "% Passing Reading", "% Overall Passing"]] = type_summary[
    ["% Passing Math", "% Passing Reading", "% Overall Passing"]
].applymap("{:.1f}%".format)

type_summary


/var/folders/21/0z_b5bn93sx2s8t9cpmzsyl40000gn/T/ipykernel_48574/2347770175.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ].applymap("{:.1f}%".format)


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,93.6%,96.6%,90.4%
District,76.9,81.0,66.6%,80.8%,53.7%
